In [5]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import unicodedata
import math
import os
import matplotlib.patches as mpatches
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import text
from tqdm import tqdm
from collections import Counter
import unicodedata
import re
import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
user = "sy91dhb"
password = "g5t49ao"
host = "178.156.142.230"
port = "5432"
db = "hd64m1ki"

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}")

In [3]:
query = "SELECT title, url, by, time, score FROM hacker_news.items WHERE type='story' AND dead IS NULL"
df = pd.read_sql(query, engine)

In [4]:
df.to_parquet("hackernews_stories.parquet")

In [6]:
class HackerNewsDataset(Dataset):
    def __init__(self, dataframe):
        self.df = dataframe.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Example: return title, score as tensor, and author
        # You can customize this as needed
        title = row['title']
        score = torch.tensor(row['score'], dtype=torch.float32)
        author = row['by']
        return {"title": title, "score": score, "author": author}

# Example usage:
hn_dataset = HackerNewsDataset(df)
hn_loader = DataLoader(hn_dataset, batch_size=32, shuffle=True)

In [7]:
# Sort the dataframe by time
df_sorted = df.sort_values('time')

# Calculate the split index
split_idx = int(len(df_sorted) * 0.8)

# Split into train and test
df_train = df_sorted.iloc[:split_idx].reset_index(drop=True)
df_test = df_sorted.iloc[split_idx:].reset_index(drop=True)

# Create datasets
hn_train = HackerNewsDataset(df_train)
hn_test = HackerNewsDataset(df_test)